In [1]:
import pandas as pd
import numpy as np

In [37]:
btc = pd.read_csv('gemini_BTCUSD_2020_1min.csv', 
                  parse_dates=['Date'], 
                  index_col='Date', 
                  usecols=['Date','Open','High','Low','Close','Volume'])
btcusd_daily = btc[['Open', 'High', 'Low', 'Close', 'Volume']].resample('D').agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last', 'Volume':'sum'})
btcusd_daily.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01,7165.90,7231.65,7146.60,7175.20,776.641615
2020-01-02,7175.20,7186.65,6901.02,6944.90,984.141603
2020-01-03,6944.90,7398.00,6860.00,7333.00,1292.330594
2020-01-04,7333.00,7394.40,7258.53,7352.82,402.636843
2020-01-05,7352.82,7490.17,7313.53,7351.77,554.773602


In [110]:
btcusd_daily.sort_index(ascending=True, inplace=True)

In [186]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, ascending=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    data.sort_index(ascending=True, inplace=True)

    columns= list(data.columns)
    
    cols, names = list(), list()
    cols.append(data)
    # input sequence (t-n, ... t-1)
    for i in range(1, n_in+1, 1):
        column_names = dict()
        
        for ii in columns:
            column_names[ii] = str(ii)+'(t-'+str(i)+')'
            
        cols.append(data.shift(i).rename(columns=column_names))

    agg = pd.concat(cols, axis=1)
    
    if dropnan:
        agg.dropna(inplace=True)
    
    if not ascending:
        agg.sort_index(ascending=False, inplace=True)
        
    return agg

In [188]:
series_to_supervised(btcusd_daily, n_in=4, dropnan=False, ascending=False)

,Open,High,Low,Close,Volume,Open(t-1),High(t-1),Low(t-1),Close(t-1),Volume(t-1),...,Open(t-3),High(t-3),Low(t-3),Close(t-3),Volume(t-3),Open(t-4),High(t-4),Low(t-4),Close(t-4),Volume(t-4)
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20,55691.79,55849.82,55541.69,55541.69,6.290095,56292.67,57624.66,54302.02,55691.79,2027.757058,...,61436.65,62578.83,59725.18,60068.01,930.943922,63237.66,63602.96,60055.14,61436.65,2584.431750
2021-04-19,56292.67,57624.66,54302.02,55691.79,2027.757058,60068.01,60443.42,50622.32,56292.67,5392.236906,...,63237.66,63602.96,60055.14,61436.65,2584.431750,62972.24,63855.12,62045.00,63237.66,1034.427743
2021-04-18,60068.01,60443.42,50622.32,56292.67,5392.236906,61436.65,62578.83,59725.18,60068.01,930.943922,...,62972.24,63855.12,62045.00,63237.66,1034.427743,63619.79,64900.00,61303.97,62972.24,3249.552490
2021-04-17,61436.65,62578.83,59725.18,60068.01,930.943922,63237.66,63602.96,60055.14,61436.65,2584.431750,...,63619.79,64900.00,61303.97,62972.24,3249.552490,59863.12,63727.00,59800.00,63619.79,1879.313409
2021-04-16,63237.66,63602.96,60055.14,61436.65,2584.431750,62972.24,63855.12,62045.00,63237.66,1034.427743,...,59863.12,63727.00,59800.00,63619.79,1879.313409,59980.60,61197.09,59400.01,59863.12,1240.628691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-05,7352.82,7490.17,7313.53,7351.77,554.773602,7333.00,7394.40,7258.53,7352.82,402.636843,...,7175.20,7186.65,6901.02,6944.90,984.141603,7165.90,7231.65,7146.60,7175.20,776.641615
2020-01-04,7333.00,7394.40,7258.53,7352.82,402.636843,6944.90,7398.00,6860.00,7333.00,1292.330594,...,7165.90,7231.65,7146.60,7175.20,776.641615,NaN,NaN,NaN,NaN,NaN
2020-01-03,6944.90,7398.00,6860.00,7333.00,1292.330594,7175.20,7186.65,6901.02,6944.90,984.141603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
series_to_supervised(btcusd_daily)

Open
High
Low
Close
Volume
None
Open
High
Low
Close
Volume
None


In [107]:
cols

[                Open      High       Low     Close       Volume
 Date                                                           
 2021-04-20  55691.79  55849.82  55541.69  55541.69     6.290095
 2021-04-19  56292.67  57624.66  54302.02  55691.79  2027.757058
 2021-04-18  60068.01  60443.42  50622.32  56292.67  5392.236906
 2021-04-17  61436.65  62578.83  59725.18  60068.01   930.943922
 2021-04-16  63237.66  63602.96  60055.14  61436.65  2584.431750
 ...              ...       ...       ...       ...          ...
 2020-01-05   7352.82   7490.17   7313.53   7351.77   554.773602
 2020-01-04   7333.00   7394.40   7258.53   7352.82   402.636843
 2020-01-03   6944.90   7398.00   6860.00   7333.00  1292.330594
 2020-01-02   7175.20   7186.65   6901.02   6944.90   984.141603
 2020-01-01   7165.90   7231.65   7146.60   7175.20   776.641615
 
 [476 rows x 5 columns]]

In [108]:
cols2

[                Open      High       Low     Close       Volume
 Date                                                           
 2021-04-20  56292.67  57624.66  54302.02  55691.79  2027.757058
 2021-04-19  60068.01  60443.42  50622.32  56292.67  5392.236906
 2021-04-18  61436.65  62578.83  59725.18  60068.01   930.943922
 2021-04-17  63237.66  63602.96  60055.14  61436.65  2584.431750
 2021-04-16  62972.24  63855.12  62045.00  63237.66  1034.427743
 ...              ...       ...       ...       ...          ...
 2020-01-05   7333.00   7394.40   7258.53   7352.82   402.636843
 2020-01-04   6944.90   7398.00   6860.00   7333.00  1292.330594
 2020-01-03   7175.20   7186.65   6901.02   6944.90   984.141603
 2020-01-02   7165.90   7231.65   7146.60   7175.20   776.641615
 2020-01-01       NaN       NaN       NaN       NaN          NaN
 
 [476 rows x 5 columns]]